<a href="https://colab.research.google.com/github/quan11103/TMDT/blob/main/Phat_hien_binh_luan_ao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Cài đặt thư viện

In [1]:
!pip install transformers[torch] datasets evaluate scikit-learn pyvi accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
   

In [2]:
import os
import re
import pandas as pd
import numpy as np
import shutil
from pyvi import ViTokenizer
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# 2. Tiền xử lí

In [3]:
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = " ".join(text.split())
    text = ViTokenizer.tokenize(text)   # Tách từ tiếng Việt
    return text

# 3. Tải dữ liệu


In [4]:
# Đọc file
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
dev_df = pd.read_csv("dev.csv")

--- Đang đọc dữ liệu từ CSV ---


In [5]:
# Chuyển sang Hugging Face Dataset
raw_datasets = DatasetDict({
    "train": Dataset.from_pandas(train_df.rename(columns={'Label': 'labels'})),
    "test": Dataset.from_pandas(test_df.rename(columns={'Label': 'labels'})),
    "validation": Dataset.from_pandas(dev_df.rename(columns={'Label': 'labels'}))
})

In [6]:
# Tải Tokenizer của PhoBERT
model_name = "vinai/phobert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
def preprocess_function(examples):
    texts = [clean_text(str(text)) for text in examples["Comment"]]
    return tokenizer(texts, padding="max_length", truncation=True, max_length=256)

print("--- Đang tiến hành Tokenize và Làm sạch ---")
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

--- Đang tiến hành Tokenize và Làm sạch ---


Map:   0%|          | 0/14306 [00:00<?, ? examples/s]

Map:   0%|          | 0/3974 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

# 4. Định nghĩa mô hình & đánh giá

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: vinai/phobert-base-v2
Key                             | Status     | 
--------------------------------+------------+-
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
classifier.dense.weight         | MISSING    | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.out_proj.weight      | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


# 5. Thiết lập huấn luyện

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)

print("--- Bắt đầu huấn luyện ---")
trainer.train()

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


--- Bắt đầu huấn luyện ---


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.299408,0.301665,0.897484,0.786928,0.906627,0.695150
2,0.192916,0.316355,0.901887,0.796875,0.913433,0.706697
3,0.152925,0.344589,0.903145,0.810811,0.866142,0.762125
4,0.109877,0.374455,0.902516,0.819558,0.826291,0.812933
5,0.076443,0.410835,0.910063,0.828331,0.862500,0.796767


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

TrainOutput(global_step=4475, training_loss=0.16114249629015362, metrics={'train_runtime': 3273.9263, 'train_samples_per_second': 21.848, 'train_steps_per_second': 1.367, 'total_flos': 9410166894950400.0, 'train_loss': 0.16114249629015362, 'epoch': 5.0})

# 6. Lưu & nén mô hình

In [11]:
print("--- Đang lưu mô hình ---")
model_save_path = "./model"
trainer.save_model(model_save_path)
tokenizer.save_pretrained(model_save_path)

--- Đang lưu mô hình ---


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

('./model/tokenizer_config.json',
 './model/vocab.txt',
 './model/bpe.codes',
 './model/added_tokens.json')

In [12]:
# Nén thư mục
shutil.make_archive("detector_model", 'zip', model_save_path)
print("Hoàn tất")

Hoàn tất
